# Food101 Dataset Classification
This notebook demonstrates a professional and simple approach to multiclass image classification using a convolutional neural network (CNN) on the Food101 dataset.

In [ ]:
pip install datasets

In [ ]:
# Import libraries
from datasets import load_dataset
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from PIL import Image
import numpy as np
from collections import Counter

In [ ]:
# Load Food101 dataset
dataset = load_dataset("food101")
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 75750
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 25250
    })
})


In [ ]:
# Create a balanced middle subset for faster training
def balanced_middle_cut(ds, num_classes=101, samples_per_class=50):
    indices = []
    class_counter = Counter()
    # Find all indices for each class
    class_indices = {i: [] for i in range(num_classes)}
    for idx, label in enumerate(ds['label']):
        class_indices[label].append(idx)
    # Select the middle samples for each class
    for label in range(num_classes):
        idxs = class_indices[label]
        if len(idxs) >= samples_per_class:
            start = (len(idxs) - samples_per_class) // 2
            indices.extend(idxs[start:start+samples_per_class])
        else:
            indices.extend(idxs)  # If not enough, take all
    return ds.select(indices)

# Use a middle-sized balanced subset for training and validation
middle_train = balanced_middle_cut(dataset['train'], samples_per_class=50)
middle_val = balanced_middle_cut(dataset['validation'], samples_per_class=25)

In [ ]:
# Preprocess images and labels
def preprocess_data(ds, img_size=(128, 128)):
    def process(example):
        image = tf.image.resize(tf.convert_to_tensor(example['image']), img_size) / 255.0
        label = tf.cast(example['label'], tf.int32)
        return {'image': image, 'label': label}
    return ds.map(process, num_proc=1)

train_data = preprocess_data(middle_train)
val_data = preprocess_data(middle_val)

train_data = train_data.to_tf_dataset(
    columns=['image'], label_cols='label', shuffle=True, batch_size=32
).prefetch(tf.data.AUTOTUNE)

val_data = val_data.to_tf_dataset(
    columns=['image'], label_cols='label', shuffle=False, batch_size=32
).prefetch(tf.data.AUTOTUNE)

Map:   0%|          | 0/5050 [00:00<?, ? examples/s]

Map:   0%|          | 0/2525 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [ ]:
# Build CNN model with MobileNetV2 backbone
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
output = Dense(101, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 64, 64,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 64, 64,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 64, 64,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 64, 64,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 64, 64,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 64, 64,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 65, 65,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 32, 32,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 32, 32,    │      2,304 │ block_1_depthwis

 Total params: 2,387,365 (9.11 MB)

 Trainable params: 129,381 (505.39 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
# Train the model
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=callbacks
)

Epoch 1/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 1078s 7s/step - accuracy: 0.0438 - loss: 5.2896 - val_accuracy: 0.2772 - val_loss: 3.0583 - learning_rate: 0.0010
Epoch 2/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 1073s 7s/step - accuracy: 0.2815 - loss: 3.0289 - val_accuracy: 0.3434 - val_loss: 2.6870 - learning_rate: 0.0010
Epoch 3/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 1094s 7s/step - accuracy: 0.4062 - loss: 2.3529 - val_accuracy: 0.3735 - val_loss: 2.5731 - learning_rate: 0.0010
Epoch 4/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 1060s 7s/step - accuracy: 0.4984 - loss: 1.9102 - val_accuracy: 0.3822 - val_loss: 2.5017 - learning_rate: 0.0010
Epoch 5/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 1101s 7s/step - accuracy: 0.5488 - loss: 1.7071 - val_accuracy: 0.3909 - val_loss: 2.5048 - learning_rate: 0.0010
Epoch 6/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 1118s 7s/step - accuracy: 0.5862 - loss: 1.5072 - val_accuracy: 0.3984 - val_loss: 2.4956 - learning_rate: 0.0010
Epoch 7/20
158/158 ━━━━━━━━━━━━━━━━━━━━ 1100s 7s/step - accuracy: 0.6388 - l

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(val_data)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

In [ ]:
# Make predictions on a new image
def preprocess_image(image_path, img_size=(128, 128)):
    image = Image.open(image_path).convert('RGB')
    image = image.resize(img_size)
    image_array = np.array(image) / 255.0
    return np.expand_dims(image_array, axis=0)

image_path = 'image_prime.jpg'
image_array = preprocess_image(image_path)
class_names = dataset['train'].features['label'].names

predictions = model.predict(image_array)
predicted_class = np.argmax(predictions, axis=1)[0]
print(f"Predicted Class: {class_names[predicted_class]}")

In [ ]:
import matplotlib.pyplot as plt

# 8. Графіки
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()

plt.tight_layout()
plt.savefig("training_curves.png")
plt.show()

In [ ]:
# Save the trained model for later use
model.save('food101_mobilenetv2.keras')

In [ ]:
from tensorflow.keras.models import load_model

loaded_model = load_model('food101_mobilenetv2.keras')

# Example: Predict with loaded model
predictions = loaded_model.predict(image_array)